In [146]:
import pandas as pd
import matplotlib.pyplot as plt

# Load DATA

In [147]:

gdpdf = pd.read_csv('https://raw.githubusercontent.com/boardkeystown/CIS568Project/main/data/API_NY.GDP.MKTP.KD.ZG_DS2_en_csv_v2_4694019.csv')
gdpdf_usd = pd.read_csv('https://raw.githubusercontent.com/boardkeystown/CIS568Project/main/data/GDP-in-USD.csv')

avg_height_men = pd.read_csv("https://raw.githubusercontent.com/boardkeystown/CIS568Project/main/data/average-height-of-men.csv")
avg_height_women = pd.read_csv("https://raw.githubusercontent.com/boardkeystown/CIS568Project/main/data/average-height-of-women.csv")

avg_height_change_women = pd.read_csv('https://raw.githubusercontent.com/boardkeystown/CIS568Project/main/data/annual-change-in-average-female-height.csv')
avg_height_change_men = pd.read_csv('https://raw.githubusercontent.com/boardkeystown/CIS568Project/main/data/annual-change-in-average-male-height.csv')


male_to_female_height_ratio = pd.read_csv('../data/male-to-female-height-ratio.csv')



countryCodesDf = pd.read_csv('https://raw.githubusercontent.com/boardkeystown/CIS568Project/main/data/wikipedia-iso-country-codes.csv')

Filter out na values, set to 0

In [148]:
male_to_female_height_ratio.head()

,Entity,Code,Year,Male-to-female height ratio
0,Afghanistan,AFG,1896,1.080277
1,Afghanistan,AFG,1897,1.079525
2,Afghanistan,AFG,1898,1.078771
3,Afghanistan,AFG,1899,1.078022
4,Afghanistan,AFG,1900,1.077272


In [149]:
gdpdf = gdpdf.fillna(0)
avg_height_men = avg_height_men.fillna(0)
avg_height_women = avg_height_women.fillna(0)
avg_height_change_women = avg_height_change_women.fillna(0)
avg_height_change_men  = avg_height_change_men.fillna(0)
gdpdf_usd = gdpdf_usd.fillna(0)

male_to_female_height_ratio = male_to_female_height_ratio.fillna(0)

# Get Data We Want To Use Into Single Dataframe

In [150]:
#copy men into new df and start with this
avg_height_human = avg_height_men.copy()

In [151]:
avg_height_human.head()

,Entity,Code,Year,Mean male height (cm)
0,Afghanistan,AFG,1896,161.164095
1,Afghanistan,AFG,1897,161.196286
2,Afghanistan,AFG,1898,161.228297
3,Afghanistan,AFG,1899,161.260727
4,Afghanistan,AFG,1900,161.293068


In [152]:
#copy over other columns

avg_height_human['Mean female height (cm)'] = avg_height_women['Mean female height (cm)'][(avg_height_human['Code'] == avg_height_women['Code']) & (avg_height_human['Year'] == avg_height_women['Year'])]
avg_height_human['change in male height'] = avg_height_change_men['Year-on-year change in male height (%)']
avg_height_human['change in female height'] = avg_height_change_women['Year-on-year change in female height (%)']


avg_height_human['Male-to-female height ratio'] = male_to_female_height_ratio['Male-to-female height ratio']

#do fill na to be safe

avg_height_human = avg_height_human.fillna(0)
avg_height_human.head()


,Entity,Code,Year,Mean male height (cm),Mean female height (cm),change in male height,change in female height,Male-to-female height ratio
0,Afghanistan,AFG,1896,161.164095,149.187747,0.01997,0.08962,1.080277
1,Afghanistan,AFG,1897,161.196286,149.321451,0.01986,0.08977,1.079525
2,Afghanistan,AFG,1898,161.228297,149.455494,0.02011,0.08966,1.078771
3,Afghanistan,AFG,1899,161.260727,149.589503,0.02005,0.08963,1.078022
4,Afghanistan,AFG,1900,161.293068,149.723587,0.02010,0.08949,1.077272


In [153]:
#take a look at GDP 

gdpdf.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,ABW,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.035375,6.429817,-0.020479,3.543630,2.116716,5.482371,1.267218,-2.085600,-22.318948,0.000000
1,Africa Eastern and Southern,AFE,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,0.0,0.239594,7.980768,5.164334,4.579952,5.327717,...,0.923067,4.200209,3.984387,2.952845,2.217017,2.557135,2.485244,2.025941,-2.886856,4.303667
2,Afghanistan,AFG,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,12.752287,5.600745,2.724543,1.451315,2.260314,2.647003,1.189228,3.911603,-2.351101,0.000000
3,Africa Western and Central,AFW,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,0.0,1.848752,3.770220,7.272440,5.396349,4.049793,...,5.142997,6.104192,5.927355,2.745894,0.127844,2.317679,2.952190,3.202544,-0.897014,3.914799
4,Angola,AGO,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,8.540000,4.950000,4.820000,0.940000,-2.580000,-0.150000,-1.316312,-0.700000,-5.500000,0.700000


Make a few functions to filter the GDP columns

In [154]:
#need to filter by columns into the avg_height_human df 
#get the years columns indexes
def filterGDPYears(df):
    gdpYears = df.columns[5:len(df.columns)-1]
    gdpYearsLookUp =  {}
    indexList = []
    yearsList = []
    for i,j in enumerate(gdpYears):
        gdpYearsLookUp[j] = i+5
        indexList.append(i+5)
        yearsList.append(j)
    return gdpYears, gdpYearsLookUp, indexList, yearsList


gdpYears, gdpYearsLookUp, indexList, yearsList = filterGDPYears(gdpdf)


In [155]:
import math

def getGDP_DFMap(df,yList,iList):
    map = {}
    for row in gdpdf.iterrows():
        if (map.get(row[1]['Country Code'],False) is False):
            s = {}
            for i,j in enumerate(iList):
                v = 0
                if not math.isnan(row[1][j]):
                    v = row[1][j]
                s |= {f"{yList[i]}":v}
            map[row[1]['Country Code']] = s
    return map

gdpdfMap_usd = getGDP_DFMap(gdpdf_usd,yearsList,indexList)

gdpdfMap = getGDP_DFMap(gdpdf,yearsList,indexList)

Now we have a way to get the info we are looking for from the gdp datasets

In [156]:
gdpdfMap.keys()

dict_keys(['ABW', 'AFE', 'AFG', 'AFW', 'AGO', 'ALB', 'AND', 'ARB', 'ARE', 'ARG', 'ARM', 'ASM', 'ATG', 'AUS', 'AUT', 'AZE', 'BDI', 'BEL', 'BEN', 'BFA', 'BGD', 'BGR', 'BHR', 'BHS', 'BIH', 'BLR', 'BLZ', 'BMU', 'BOL', 'BRA', 'BRB', 'BRN', 'BTN', 'BWA', 'CAF', 'CAN', 'CEB', 'CHE', 'CHI', 'CHL', 'CHN', 'CIV', 'CMR', 'COD', 'COG', 'COL', 'COM', 'CPV', 'CRI', 'CSS', 'CUB', 'CUW', 'CYM', 'CYP', 'CZE', 'DEU', 'DJI', 'DMA', 'DNK', 'DOM', 'DZA', 'EAP', 'EAR', 'EAS', 'ECA', 'ECS', 'ECU', 'EGY', 'EMU', 'ERI', 'ESP', 'EST', 'ETH', 'EUU', 'FCS', 'FIN', 'FJI', 'FRA', 'FRO', 'FSM', 'GAB', 'GBR', 'GEO', 'GHA', 'GIB', 'GIN', 'GMB', 'GNB', 'GNQ', 'GRC', 'GRD', 'GRL', 'GTM', 'GUM', 'GUY', 'HIC', 'HKG', 'HND', 'HPC', 'HRV', 'HTI', 'HUN', 'IBD', 'IBT', 'IDA', 'IDB', 'IDN', 'IDX', 'IMN', 'IND', 'INX', 'IRL', 'IRN', 'IRQ', 'ISL', 'ISR', 'ITA', 'JAM', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ', 'KHM', 'KIR', 'KNA', 'KOR', 'KWT', 'LAC', 'LAO', 'LBN', 'LBR', 'LBY', 'LCA', 'LCN', 'LDC', 'LIC', 'LIE', 'LKA', 'LMC', 'LMY', 'L

In [157]:
gdpdfMap["JPN"]

{'1961': 12.04353641,
 '1962': 8.908972996,
 '1963': 8.473642383,
 '1964': 11.6767082,
 '1965': 5.819707873,
 '1966': 10.63856156,
 '1967': 11.08214232,
 '1968': 12.88246814,
 '1969': 12.47789451,
 '1970': 2.454958384,
 '1971': 4.698992042,
 '1972': 8.413547255,
 '1973': 8.032599975,
 '1974': -1.225239828,
 '1975': 3.091575916,
 '1976': 3.974984091,
 '1977': 4.39033795,
 '1978': 5.271941503,
 '1979': 5.484041832,
 '1980': 2.817591208,
 '1981': 4.260624493,
 '1982': 3.279742743,
 '1983': 3.630198572,
 '1984': 4.410880154,
 '1985': 5.159807905,
 '1986': 3.294047613,
 '1987': 4.648861398,
 '1988': 6.661636522,
 '1989': 4.925785413,
 '1990': 4.840929057,
 '1991': 3.523357235,
 '1992': 0.900586066,
 '1993': -0.459219716,
 '1994': 1.083383174,
 '1995': 2.630999616,
 '1996': 3.133870993,
 '1997': 0.981228732,
 '1998': -1.270330495,
 '1999': -0.333929958,
 '2000': 2.764647551,
 '2001': 0.386103426,
 '2002': 0.041962499,
 '2003': 1.535125499,
 '2004': 2.186115694,
 '2005': 1.803900872,
 '2006':

Need a list of country codes

In [158]:
ccode = gdpdf['Country Code']
ccode

0      ABW
1      AFE
2      AFG
3      AFW
4      AGO
      ... 
261    XKX
262    YEM
263    ZAF
264    ZMB
265    ZWE
Name: Country Code, Length: 266, dtype: object

Now need to figure out what was in the map so that we can add to each row of the avg_height_human df if we have the data for it.

In [159]:
def makeGDPColumn(df,map):
    columnGDPKeep = []
    for row in avg_height_human.iterrows():
        v = 0
        if (map.get(code := row[1]['Code'],False)):
            if (map[code].get(year := str(row[1]['Year']),False)):
                v = map[code][year]
        columnGDPKeep.append(v)
    return columnGDPKeep

GDPKeep = makeGDPColumn(avg_height_human,gdpdfMap)

GDPKeep_usd = makeGDPColumn(avg_height_human,gdpdfMap_usd)

In [160]:
avg_height_human["GDP annual growth"] = GDPKeep
avg_height_human.head(3)

,Entity,Code,Year,Mean male height (cm),Mean female height (cm),change in male height,change in female height,Male-to-female height ratio,GDP annual growth
0,Afghanistan,AFG,1896,161.164095,149.187747,0.01997,0.08962,1.080277,0.0
1,Afghanistan,AFG,1897,161.196286,149.321451,0.01986,0.08977,1.079525,0.0
2,Afghanistan,AFG,1898,161.228297,149.455494,0.02011,0.08966,1.078771,0.0


In [161]:
avg_height_human["GDP annual USD"] = GDPKeep_usd
avg_height_human.tail(3)

,Entity,Code,Year,Mean male height (cm),Mean female height (cm),change in male height,change in female height,Male-to-female height ratio,GDP annual growth,GDP annual USD
21005,Zimbabwe,ZWE,1994,168.963595,158.400025,0.0,0.0,1.066689,9.235199,9.235199
21006,Zimbabwe,ZWE,1995,168.774934,158.307779,0.0,0.0,1.066119,0.158026,0.158026
21007,Zimbabwe,ZWE,1996,168.586005,158.215186,0.0,0.0,1.065549,10.360697,10.360697


In [162]:
avg_height_human.tail(3)

,Entity,Code,Year,Mean male height (cm),Mean female height (cm),change in male height,change in female height,Male-to-female height ratio,GDP annual growth,GDP annual USD
21005,Zimbabwe,ZWE,1994,168.963595,158.400025,0.0,0.0,1.066689,9.235199,9.235199
21006,Zimbabwe,ZWE,1995,168.774934,158.307779,0.0,0.0,1.066119,0.158026,0.158026
21007,Zimbabwe,ZWE,1996,168.586005,158.215186,0.0,0.0,1.065549,10.360697,10.360697


Add the two letter country codes!

In [163]:
countryCodesMap = {}
def makeCountryCodesMap(df):
    _map = {}
    Name=0;Alpha2=1;Alpha3=2;Numeric=3
    for row in df.iterrows():
        # print(row[1])
        county,alpha2,alpha3,numeric,na = row[1]
        _map[alpha3] = {"alpha2":alpha2,"Number":numeric}
    return _map
countryCodesMap=makeCountryCodesMap(countryCodesDf)

In [164]:
# countryCodes.head()
columnAlpha2Code = []
def makeCountryCodes3To2column(map,df):
    for row in df.iterrows():
        # print(row)
        # code = row[1][1]
        alpha2_code = "N/A"
        if (map.get(code := row[1][1],False)):
            alpha2_code =  map[code]["alpha2"]
        columnAlpha2Code.append(alpha2_code)
    return columnAlpha2Code

columnAlpha2Code = makeCountryCodes3To2column(countryCodesMap,avg_height_human)

In [165]:
avg_height_human.insert(2,"code_alpha2",columnAlpha2Code)


In [166]:
avg_height_human.head()

,Entity,Code,code_alpha2,Year,Mean male height (cm),Mean female height (cm),change in male height,change in female height,Male-to-female height ratio,GDP annual growth,GDP annual USD
0,Afghanistan,AFG,AF,1896,161.164095,149.187747,0.01997,0.08962,1.080277,0.0,0.0
1,Afghanistan,AFG,AF,1897,161.196286,149.321451,0.01986,0.08977,1.079525,0.0,0.0
2,Afghanistan,AFG,AF,1898,161.228297,149.455494,0.02011,0.08966,1.078771,0.0,0.0
3,Afghanistan,AFG,AF,1899,161.260727,149.589503,0.02005,0.08963,1.078022,0.0,0.0
4,Afghanistan,AFG,AF,1900,161.293068,149.723587,0.02010,0.08949,1.077272,0.0,0.0


In [167]:
print(f"         gdp years: max - min:{gdpYears.max()},{gdpYears.min()}")
print(f"height_human years: max - min:{avg_height_human['Year'].max()},{avg_height_human['Year'].min()}")

         gdp years: max - min:2020,1961
height_human years: max - min:1996,1896


The human height index dataset has years then we have gdp data for. So we should remove years we do not have.

In [168]:
dropDf = avg_height_human[(avg_height_human['Year'] < 1960)].index
avg_height_human.drop(dropDf , inplace=True)

In [169]:
avg_height_human["GDP annual growth"].describe()

count    7696.000000
mean        2.388765
std         5.569407
min       -64.047107
25%         0.000000
50%         0.000000
75%         4.943125
max        88.957666
Name: GDP annual growth, dtype: float64

In [170]:
for row in avg_height_human.iterrows():
    if (row[1]["GDP annual growth"] > 50):
        print(row)

(2423, Entity                         Bosnia and Herzegovina
Code                                              BIH
code_alpha2                                        BA
Year                                             1996
Mean male height (cm)                      180.869174
Mean female height (cm)                     165.84831
change in male height                         0.05023
change in female height                       0.06966
Male-to-female height ratio                   1.09057
GDP annual growth                           88.957666
GDP annual USD                              88.957666
Name: 2423, dtype: object)
(5857, Entity                         Equatorial Guinea
Code                                         GNQ
code_alpha2                                   GQ
Year                                        1996
Mean male height (cm)                 167.362147
Mean female height (cm)               157.326344
change in male height                    0.02882
change in female heigh

In [171]:
avg_height_human["code_alpha2"].unique()

array(['AF', 'AL', 'DZ', 'AS', 'AD', 'AO', 'AG', 'AR', 'AM', 'AU', 'AT',
       'AZ', 'BS', 'BH', 'BD', 'BB', 'BY', 'BE', 'BZ', 'BJ', 'BM', 'BT',
       'BO', 'BA', 'BW', 'BR', 'BN', 'BG', 'BF', 'BI', 'KH', 'CM', 'CA',
       'CV', 'CF', 'TD', 'CL', 'CN', 'CO', 'KM', 'CG', 'CK', 'CR', 'CI',
       'HR', 'CU', 'CY', 'CZ', 'CD', 'DK', 'DJ', 'DM', 'DO', 'N/A', 'EC',
       'EG', 'SV', 'GQ', 'ER', 'EE', 'SZ', 'ET', 'FJ', 'FI', 'FR', 'PF',
       'GA', 'GM', 'GE', 'DE', 'GH', 'GR', 'GL', 'GD', 'GT', 'GN', 'GW',
       'GY', 'HT', 'HN', 'HK', 'HU', 'IS', 'IN', 'ID', 'IR', 'IQ', 'IE',
       'IL', 'IT', 'JM', 'JP', 'JO', 'KZ', 'KE', 'KI', 'KW', 'KG', 'LA',
       'LV', 'LB', 'LS', 'LR', 'LY', 'LT', 'LU', 'MG', 'MW', 'MY', 'MV',
       'ML', 'MT', 'MH', 'MR', 'MU', 'MX', 'FM', 'MD', 'MN', 'ME', 'MA',
       'MZ', 'MM', nan, 'NR', 'NP', 'NL', 'NZ', 'NI', 'NE', 'NG', 'NU',
       'KP', 'MK', 'NO', 'OM', 'PK', 'PW', 'PS', 'PA', 'PG', 'PY', 'PE',
       'PH', 'PL', 'PT', 'PR', 'QA', 'RO', 'RU', 'R

In [172]:
avg_height_human["Code"].unique()

array(['AFG', 'ALB', 'DZA', 'ASM', 'AND', 'AGO', 'ATG', 'ARG', 'ARM',
       'AUS', 'AUT', 'AZE', 'BHS', 'BHR', 'BGD', 'BRB', 'BLR', 'BEL',
       'BLZ', 'BEN', 'BMU', 'BTN', 'BOL', 'BIH', 'BWA', 'BRA', 'BRN',
       'BGR', 'BFA', 'BDI', 'KHM', 'CMR', 'CAN', 'CPV', 'CAF', 'TCD',
       'CHL', 'CHN', 'COL', 'COM', 'COG', 'COK', 'CRI', 'CIV', 'HRV',
       'CUB', 'CYP', 'CZE', 'COD', 'DNK', 'DJI', 'DMA', 'DOM', 0, 'ECU',
       'EGY', 'SLV', 'GNQ', 'ERI', 'EST', 'SWZ', 'ETH', 'FJI', 'FIN',
       'FRA', 'PYF', 'GAB', 'GMB', 'GEO', 'DEU', 'GHA', 'GRC', 'GRL',
       'GRD', 'GTM', 'GIN', 'GNB', 'GUY', 'HTI', 'HND', 'HKG', 'HUN',
       'ISL', 'IND', 'IDN', 'IRN', 'IRQ', 'IRL', 'ISR', 'ITA', 'JAM',
       'JPN', 'JOR', 'KAZ', 'KEN', 'KIR', 'KWT', 'KGZ', 'LAO', 'LVA',
       'LBN', 'LSO', 'LBR', 'LBY', 'LTU', 'LUX', 'MDG', 'MWI', 'MYS',
       'MDV', 'MLI', 'MLT', 'MHL', 'MRT', 'MUS', 'MEX', 'FSM', 'MDA',
       'MNG', 'MNE', 'MAR', 'MOZ', 'MMR', 'NAM', 'NRU', 'NPL', 'NLD',
       'NZL', 'NI

In [173]:
avg_height_human.head(3)

,Entity,Code,code_alpha2,Year,Mean male height (cm),Mean female height (cm),change in male height,change in female height,Male-to-female height ratio,GDP annual growth,GDP annual USD
64,Afghanistan,AFG,AF,1960,164.742777,156.404249,0.03299,0.02085,1.053314,0.0,0.0
65,Afghanistan,AFG,AF,1961,164.797132,156.436857,0.03198,0.01791,1.053442,0.0,0.0
66,Afghanistan,AFG,AF,1962,164.849830,156.464874,0.02991,0.01378,1.053590,0.0,0.0


NOW LETS SAVE THE DATASET TO CSV

In [174]:

avg_height_human.to_csv('./avg_height_human_country_gdp.csv', index=False)

# avg_height_human.to_csv('../data/avg_height_human_country_gdp.csv', index=False)
import os

os.system('cp ./avg_height_human_country_gdp.csv ../data/')


1

# Lets look at the data using python

In [175]:
import numpy as np  ## Numpy is one of the default data science libraries, it contains numerous functions that help analyze and reshaping data
# Matplotlib
import matplotlib.pyplot as plt ## matplotlib is the default visualization library for python 

In [176]:
def getDFByCode(code,dfin):
    df_temp = pd.DataFrame(columns=['Year','GDP annual growth','change in male height','change in female height'])
    df_temp['Year'] = dfin['Year'][(dfin['Code']==code)]
    df_temp['GDP annual growth'] = dfin['GDP annual growth'][dfin['Code']==code]
    df_temp['GDP annual USD'] = dfin['GDP annual USD'][dfin['Code']==code]
    df_temp['change in male height'] = dfin['change in male height'][dfin['Code']==code]
    df_temp['change in female height'] = dfin['change in female height'][dfin['Code']==code]
    df_temp['Mean male height (cm)'] = dfin['Mean male height (cm)'][dfin['Code']==code]
    df_temp['Mean female height (cm)'] = dfin['Mean female height (cm)'][dfin['Code']==code]
    df_temp['Male-to-female height ratio'] = dfin['Male-to-female height ratio'][dfin['Code']==code]
    return df_temp

In [177]:
corrDF = getDFByCode("CHL",avg_height_human)

In [178]:
corr = corrDF.corr(method='kendall')
corr.style.background_gradient(cmap='coolwarm')

,Year,GDP annual growth,change in male height,change in female height,GDP annual USD,Mean male height (cm),Mean female height (cm),Male-to-female height ratio
Year,1.000000,0.234234,-0.945946,0.447447,0.234234,1.000000,1.000000,-0.645646
GDP annual growth,0.234234,1.000000,-0.216216,0.264264,1.000000,0.234234,0.234234,-0.216216
change in male height,-0.945946,-0.216216,1.000000,-0.393393,-0.216216,-0.945946,-0.945946,0.597598
change in female height,0.447447,0.264264,-0.393393,1.000000,0.264264,0.447447,0.447447,-0.627628
GDP annual USD,0.234234,1.000000,-0.216216,0.264264,1.000000,0.234234,0.234234,-0.216216
Mean male height (cm),1.000000,0.234234,-0.945946,0.447447,0.234234,1.000000,1.000000,-0.645646
Mean female height (cm),1.000000,0.234234,-0.945946,0.447447,0.234234,1.000000,1.000000,-0.645646
Male-to-female height ratio,-0.645646,-0.216216,0.597598,-0.627628,-0.216216,-0.645646,-0.645646,1.000000


In [179]:
corrDF = getDFByCode("PAK",avg_height_human).copy()
corr = corrDF.corr(method='kendall')
corr.style.background_gradient(cmap='coolwarm')

,Year,GDP annual growth,change in male height,change in female height,GDP annual USD,Mean male height (cm),Mean female height (cm),Male-to-female height ratio
Year,1.000000,-0.099099,-0.246246,-0.585404,-0.099099,-0.948949,-1.000000,1.000000
GDP annual growth,-0.099099,1.000000,0.066066,0.064710,1.000000,0.132132,0.099099,-0.099099
change in male height,-0.246246,0.066066,1.000000,0.624531,0.066066,0.225225,0.246246,-0.246246
change in female height,-0.585404,0.064710,0.624531,1.000000,0.064710,0.534238,0.585404,-0.585404
GDP annual USD,-0.099099,1.000000,0.066066,0.064710,1.000000,0.132132,0.099099,-0.099099
Mean male height (cm),-0.948949,0.132132,0.225225,0.534238,0.132132,1.000000,0.948949,-0.948949
Mean female height (cm),-1.000000,0.099099,0.246246,0.585404,0.099099,0.948949,1.000000,-1.000000
Male-to-female height ratio,1.000000,-0.099099,-0.246246,-0.585404,-0.099099,-0.948949,-1.000000,1.000000


In [180]:
corrDF = getDFByCode("USA",avg_height_human).copy()
corr = corrDF.corr(method='kendall')
corr.style.background_gradient(cmap='coolwarm')

,Year,GDP annual growth,change in male height,change in female height,GDP annual USD,Mean male height (cm),Mean female height (cm),Male-to-female height ratio
Year,1.000000,-0.126126,-0.993994,-0.993994,-0.126126,-0.051051,-0.660661,0.423423
GDP annual growth,-0.126126,1.000000,0.126126,0.132132,1.000000,0.042042,0.099099,-0.102102
change in male height,-0.993994,0.126126,1.000000,0.987988,0.126126,0.051051,0.654655,-0.423423
change in female height,-0.993994,0.132132,0.987988,1.000000,0.132132,0.045045,0.654655,-0.417417
GDP annual USD,-0.126126,1.000000,0.126126,0.132132,1.000000,0.042042,0.099099,-0.102102
Mean male height (cm),-0.051051,0.042042,0.051051,0.045045,0.042042,1.000000,0.390390,0.141141
Mean female height (cm),-0.660661,0.099099,0.654655,0.654655,0.099099,0.390390,1.000000,-0.468468
Male-to-female height ratio,0.423423,-0.102102,-0.423423,-0.417417,-0.102102,0.141141,-0.468468,1.000000


In [181]:
corrDF = getDFByCode("JPN",avg_height_human).copy()
corr = corrDF.corr(method='spearman')
corr.style.background_gradient(cmap='gist_ncar')

,Year,GDP annual growth,change in male height,change in female height,GDP annual USD,Mean male height (cm),Mean female height (cm),Male-to-female height ratio
Year,1.000000,-0.543623,-0.996147,-0.985775,-0.543623,0.269322,0.429587,-0.746562
GDP annual growth,-0.543623,1.000000,0.542944,0.512802,1.000000,-0.198198,-0.318634,0.339497
change in male height,-0.996147,0.542944,1.000000,0.981922,0.542944,-0.262818,-0.425701,0.733566
change in female height,-0.985775,0.512802,0.981922,1.000000,0.512802,-0.293978,-0.447606,0.732338
GDP annual USD,-0.543623,1.000000,0.542944,0.512802,1.000000,-0.198198,-0.318634,0.339497
Mean male height (cm),0.269322,-0.198198,-0.262818,-0.293978,-0.198198,1.000000,0.963727,0.060455
Mean female height (cm),0.429587,-0.318634,-0.425701,-0.447606,-0.318634,0.963727,1.000000,-0.035562
Male-to-female height ratio,-0.746562,0.339497,0.733566,0.732338,0.339497,0.060455,-0.035562,1.000000


In [182]:
corrDF = getDFByCode("FRA",avg_height_human).copy()
corr = corrDF.corr(method='spearman')
corr.style.background_gradient(cmap='gist_ncar')

,Year,GDP annual growth,change in male height,change in female height,GDP annual USD,Mean male height (cm),Mean female height (cm),Male-to-female height ratio
Year,1.000000,-0.627549,0.876956,0.457854,-0.627549,1.000000,1.000000,1.000000
GDP annual growth,-0.627549,1.000000,-0.588668,-0.503379,1.000000,-0.627549,-0.627549,-0.627549
change in male height,0.876956,-0.588668,1.000000,0.698162,-0.588668,0.876956,0.876956,0.876956
change in female height,0.457854,-0.503379,0.698162,1.000000,-0.503379,0.457854,0.457854,0.457854
GDP annual USD,-0.627549,1.000000,-0.588668,-0.503379,1.000000,-0.627549,-0.627549,-0.627549
Mean male height (cm),1.000000,-0.627549,0.876956,0.457854,-0.627549,1.000000,1.000000,1.000000
Mean female height (cm),1.000000,-0.627549,0.876956,0.457854,-0.627549,1.000000,1.000000,1.000000
Male-to-female height ratio,1.000000,-0.627549,0.876956,0.457854,-0.627549,1.000000,1.000000,1.000000
